<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/Sequential_Chain%5B%E6%89%B9%E9%87%8F%E6%B5%81%E7%A8%8B%E4%B8%80%E6%AC%A1%E6%80%A7%E5%9B%9E%E5%A4%8D%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sequential Chain是LangChain库中的一个强大工具，它允许我们将多个LLMChain按照特定的顺序连接起来，形成一个处理流程。这种链式结构使得我们可以将一个大任务分解为几个小任务，并依次执行，每个任务的输出成为下一个任务的输入。

重点：任务分解，依次执行，任务的输出成为下一个任务的输入

In [1]:
import os
import getpass

api_key = os.getenv("ZHIPUAI_API_KEY")
if api_key is None:
  os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

··········


In [19]:
#!pip install --upgrade --quiet  langchain-google-genai pillow
#!pip install langchain langchain_community
#!pip install --upgrade httpx httpx-sse PyJWT

from langchain_community.chat_models import ChatZhipuAI

from langchain_community.chat_message_histories import ChatMessageHistory
#from langchain_core.chat_history import BaseChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.chains import ConversationChain

#os.environ.setdefault('ZHIPUAI_API_KEY', 'key')
#import os
#import getpass
#os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
)

In [20]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
#第一个模版定义
one_template = """
你是一个植物学家。给定花的名称和类型，你需要为这种花写一个200字左右的介绍。
花名: {name}颜色: {color}
植物学家: 这是关于上述花的介绍:
"""
one_prompt_template = PromptTemplate(
    template=one_template,
    input_variables=["name", "color"],
)
introduction_chain = LLMChain(
    llm=llm,
    prompt=one_prompt_template,
    output_key="introduction",
)


In [21]:
#第2个模版定义
two_template = """
你是一位鲜花评论家。给定一种花的介绍，你需要为这种花写一篇200字左右的评论。
鲜花介绍:{introduction}
花评人对上述花的评论:
"""
two_prompt_template = PromptTemplate(
    template=two_template,
    input_variables=["introduction"],
)
review_chain = LLMChain(
    llm=llm,
    prompt=two_prompt_template,
    output_key="review",
)

In [22]:
#创建第三个LLMChain：
#根据鲜花的介绍和评论撰写社交媒体文案。
three_template = """
你是一家花店的社交媒体经理。给定一种花的介绍和评论，你需要为这种花写一篇社交媒体的帖子，300字左右。
鲜花介绍:{introduction}花评人对上述花的评论:{review}
社交媒体帖子:
"""
three_prompt_template = PromptTemplate(
    template=three_template,
    input_variables=["introduction", "review"],
)
social_media_chain = LLMChain(
    llm=llm,
    prompt=three_prompt_template,
    output_key="social_post_text",
)

In [23]:
#将前面3个链串起来。
overall_chain = SequentialChain(
    chains=[introduction_chain, review_chain,social_media_chain],
    input_variables=["name", "color"],
    output_variables=["introduction", "review", "social_post_text"],
    verbose=True
)

In [24]:
#调用SequentialChain
overall_chain({"name": "玫瑰", "color": "红色"})



> Entering new SequentialChain chain...

> Finished chain.


{'name': '玫瑰',
 'color': '红色',
 'introduction': '玫瑰，被誉为“花中皇后”，其红色品种尤为著名，象征着热情与浪漫。红色玫瑰，花形优美，花瓣重叠，色彩鲜艳，给人一种强烈的视觉冲击。这种花原产于中国，后传入欧洲，逐渐成为全球最受欢迎的花卉之一。在植物学上，红色玫瑰属于蔷薇科蔷薇属，是一种具有浓郁香气的观赏植物。其花期较长，耐修剪，易繁殖，适合作为园艺植物养护。此外，红色玫瑰还具有一定的药用价值，可以提炼精油，制作护肤品和香水。在文化意义上，红色玫瑰常被用来表达爱意和敬意，是爱情、浪漫、幸福的象征。赠予他人红色玫瑰，意味着传递美好的祝福和真挚的情感。',
 'review': '红色玫瑰，这一花中皇后，以其热情如火的姿态和浪漫深邃的内涵，自古以来便是赞美的焦点。花形优雅，花瓣层叠似锦，那鲜艳的红色不仅仅是一抹色彩，更是一股强烈的情感冲击。源自我国的这片红色浪漫，已在全球范围内绽放其独特魅力。在园艺中，其花期之长、易养护的特性，使其成为众多爱花人士的心头好。而其药用价值及在提炼精油等方面的妙用，更是让人惊叹不已。红色玫瑰不仅是爱情的象征，它传递的，还有敬意、幸福和美好的祝愿。赠人玫瑰，手留余香，这朵红色玫瑰，无疑是最能表达深情厚意的选择。',
 'social_post_text': '🌹【#红色玫瑰# 花中皇后，热情浪漫的传递者】🌹\n\n她是花中的皇后，热情与浪漫的化身——红色玫瑰。源自我国的这片红色浪漫，以其优雅的花形，层叠似锦的花瓣，鲜艳夺目的色彩，传递着强烈的情感冲击。🔥\n\n红色玫瑰，不仅是蔷薇科蔷薇属的一员，更是园艺中的宠儿，爱花人士的心头好。她的花期长，易养护，还有着丰富的药用价值和提炼精油的应用，让人惊叹不已。💐\n\n在文化意义上，红色玫瑰象征着爱情、浪漫、幸福，传递着敬意和美好的祝愿。赠予他人，不仅是深情厚意的表达，更是传递美好祝福的使者。🎁\n\n"赠人玫瑰，手留余香"，这朵红色玫瑰，是你表达爱意、传递情感的绝佳选择。在这个充满浪漫的日子里，让我们一起用红色玫瑰，传递心中的热情与爱意吧！🌹\n\n#爱情象征# #浪漫传递# #红色浪漫# #花店推荐#'}

{'name': '玫瑰',
 'color': '红色',

 'introduction': '玫瑰，被誉为“花中皇后”，其红色品种尤为著名，象征着热情与浪漫。红色玫瑰，花形优美，花瓣重叠，色彩鲜艳，给人一种强烈的视觉冲击。这种花原产于中国，后传入欧洲，逐渐成为全球最受欢迎的花卉之一。在植物学上，红色玫瑰属于蔷薇科蔷薇属，是一种具有浓郁香气的观赏植物。其花期较长，耐修剪，易繁殖，适合作为园艺植物养护。此外，红色玫瑰还具有一定的药用价值，可以提炼精油，制作护肤品和香水。在文化意义上，红色玫瑰常被用来表达爱意和敬意，是爱情、浪漫、幸福的象征。赠予他人红色玫瑰，意味着传递美好的祝福和真挚的情感。',

 'review': '红色玫瑰，这一花中皇后，以其热情如火的姿态和浪漫深邃的内涵，自古以来便是赞美的焦点。花形优雅，花瓣层叠似锦，那鲜艳的红色不仅仅是一抹色彩，更是一股强烈的情感冲击。源自我国的这片红色浪漫，已在全球范围内绽放其独特魅力。在园艺中，其花期之长、易养护的特性，使其成为众多爱花人士的心头好。而其药用价值及在提炼精油等方面的妙用，更是让人惊叹不已。红色玫瑰不仅是爱情的象征，它传递的，还有敬意、幸福和美好的祝愿。赠人玫瑰，手留余香，这朵红色玫瑰，无疑是最能表达深情厚意的选择。',

 'social_post_text': '🌹【#红色玫瑰# 花中皇后，热情浪漫的传递者】🌹
 她是花中的皇后，热情与浪漫的化身——红色玫瑰。源自我国的这片红色浪漫，以其优雅的花形，层叠似锦的花瓣，鲜艳夺目的色彩，传递着强烈的情感冲击。🔥
 红色玫瑰，不仅是蔷薇科蔷薇属的一员，更是园艺中的宠儿，爱花人士的心头好。她的花期长，易养护，还有着丰富的药用价值和提炼精油的应用，让人惊叹不已。💐
 在文化意义上，红色玫瑰象征着爱情、浪漫、幸福，传递着敬意和美好的祝愿。赠予他人，不仅是深情厚意的表达，更是传递美好祝福的使者。🎁
 "赠人玫瑰，手留余香"，这朵红色玫瑰，是你表达爱意、传递情感的绝佳选择。在这个充满浪漫的日子里，让我们一起用红色玫瑰，传递心中的热情与爱意吧！🌹
 #爱情象征# #浪漫传递# #红色浪漫# #花店推荐#'}

